In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from scipy.stats import randint, uniform
from time import time


In [ ]:
from sklearn.datasets import make_classification

In [4]:
# Генерация данных
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_classes=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Поиска гиперпараметров
def show_cv_info(cv):
    df_results = pd.DataFrame(cv.cv_results_)
    print(f"Лучшие параметры: {cv.best_params_}")
    print(f"Лучшая точность: {cv.best_score_}")
    return df_results.sort_values("rank_test_score").head(3)

In [6]:
# Поиск по решетке (Grid Search)
grid_params = {
    "n_estimators": [10, 100, 500],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
}
print("Grid Search...")
start_time = time()
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), grid_params, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
grid_duration = time() - start_time
grid_results = show_cv_info(grid_search)

Grid Search...
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Лучшие параметры: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 500}
Лучшая точность: 0.7428927772275412


In [7]:
# Случайный поиск (Random Search)
random_params = {
    "n_estimators": randint(10, 500),
    "max_depth": randint(5, 50),
    "min_samples_split": randint(2, 10),
}
print("\nRandom Search...")
start_time = time()
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), random_params, cv=3, n_jobs=-1, n_iter=50, verbose=1, random_state=42)
random_search.fit(X_train, y_train)
random_duration = time() - start_time
random_results = show_cv_info(random_search)


Random Search...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Лучшие параметры: {'max_depth': 43, 'min_samples_split': 5, 'n_estimators': 358}
Лучшая точность: 0.747178508981084


In [8]:
# Байесовский поиск (Bayes Search)
bayes_params = {
    "n_estimators": Integer(10, 500),
    "max_depth": Integer(5, 50),
    "min_samples_split": Integer(2, 10),
}
print("\nBayes Search...")
start_time = time()
bayes_search = BayesSearchCV(RandomForestClassifier(random_state=42), bayes_params, cv=3, n_jobs=-1, n_iter=30, verbose=1, random_state=42)
bayes_search.fit(X_train, y_train)
bayes_duration = time() - start_time
bayes_results = show_cv_info(bayes_search)


Bayes Search...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidate

In [9]:
# Оценка результатов
print("\nРезультаты на тестовых данных:")
models = {
    "Grid Search": grid_search,
    "Random Search": random_search,
    "Bayes Search": bayes_search,
}
for name, model in models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name}: Точность = {acc:.4f}")


Результаты на тестовых данных:
Grid Search: Точность = 0.7667
Random Search: Точность = 0.7700
Bayes Search: Точность = 0.7733


In [10]:
# Сравнение времени выполнения
print("\nВремя выполнения:")
print(f"Grid Search: {grid_duration:.2f} секунд")
print(f"Random Search: {random_duration:.2f} секунд")
print(f"Bayes Search: {bayes_duration:.2f} секунд")


Время выполнения:
Grid Search: 46.41 секунд
Random Search: 67.37 секунд
Bayes Search: 130.11 секунд
